# Hierarchical Model
In this apart, we go to explain how is the finally of us Dataframe 
First: we have a selection regarding Amazon, because its data is interesting.
we have 13 columns where need show a model hierarchical
What are we going to need


In [12]:
# --- Procesamiento y Limpieza ---
import pandas as pd
import numpy as np
import polars as pl
import polars.selectors as cs
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import BayesianRidge # La opción fácil

# --- Modelado Bayesiano ---
import pymc as pm
import arviz as az



# --- Visualización ---
import matplotlib.pyplot as plt
import seaborn as sns


# La ruta entra a la carpeta 'dataset' y luego busca el archivo
df = pl.read_csv("../dataset/amazon_sales_dataset.csv")
# Para mi modelo jerarquico necesito transformar la columna de categorias (hoga, electronica a int)

print(df)


shape: (50_000, 13)
┌──────────┬────────────┬────────────┬────────────┬───┬────────┬───────────┬───────────┬───────────┐
│ order_id ┆ order_date ┆ product_id ┆ product_ca ┆ … ┆ rating ┆ review_co ┆ discounte ┆ total_rev │
│ ---      ┆ ---        ┆ ---        ┆ tegory     ┆   ┆ ---    ┆ unt       ┆ d_price   ┆ enue      │
│ i64      ┆ str        ┆ i64        ┆ ---        ┆   ┆ f64    ┆ ---       ┆ ---       ┆ ---       │
│          ┆            ┆            ┆ str        ┆   ┆        ┆ i64       ┆ f64       ┆ f64       │
╞══════════╪════════════╪════════════╪════════════╪═══╪════════╪═══════════╪═══════════╪═══════════╡
│ 1        ┆ 2022-04-13 ┆ 2637       ┆ Books      ┆ … ┆ 3.5    ┆ 443       ┆ 115.88    ┆ 463.52    │
│ 2        ┆ 2023-03-12 ┆ 2300       ┆ Fashion    ┆ … ┆ 3.7    ┆ 475       ┆ 242.08    ┆ 1210.4    │
│ 3        ┆ 2022-09-28 ┆ 3670       ┆ Sports     ┆ … ┆ 4.4    ┆ 183       ┆ 396.64    ┆ 793.28    │
│ 4        ┆ 2022-04-17 ┆ 2522       ┆ Books      ┆ … ┆ 5.0    ┆ 212   

In [13]:
# Por que elijo estas columnas ? 

columnas_interesantes = [
    "product_category",
    "price",
    "total_revenue",
    "rating"
    
]  

# Creando dataframe de trabajo 
df_modelJ = df.select(columnas_interesantes)

print(df_modelJ)

shape: (50_000, 4)
┌──────────────────┬────────┬───────────────┬────────┐
│ product_category ┆ price  ┆ total_revenue ┆ rating │
│ ---              ┆ ---    ┆ ---           ┆ ---    │
│ str              ┆ f64    ┆ f64           ┆ f64    │
╞══════════════════╪════════╪═══════════════╪════════╡
│ Books            ┆ 128.75 ┆ 463.52        ┆ 3.5    │
│ Fashion          ┆ 302.6  ┆ 1210.4        ┆ 3.7    │
│ Sports           ┆ 495.8  ┆ 793.28        ┆ 4.4    │
│ Books            ┆ 371.95 ┆ 1264.64       ┆ 5.0    │
│ Beauty           ┆ 201.68 ┆ 806.72        ┆ 4.6    │
│ …                ┆ …      ┆ …             ┆ …      │
│ Beauty           ┆ 26.99  ┆ 134.95        ┆ 2.4    │
│ Beauty           ┆ 294.23 ┆ 1324.05       ┆ 3.1    │
│ Electronics      ┆ 352.11 ┆ 985.92        ┆ 3.1    │
│ Beauty           ┆ 307.54 ┆ 292.16        ┆ 1.8    │
│ Home & Kitchen   ┆ 253.44 ┆ 177.41        ┆ 2.1    │
└──────────────────┴────────┴───────────────┴────────┘


In [14]:
# Esto es lo más importante, para ver las escalas, minimos, maximos 
df_modelJ.describe()

statistic,product_category,price,total_revenue,rating
str,str,f64,f64,f64
"""count""","""50000""",50000.0,50000.0,50000.0
"""null_count""","""0""",0.0,0.0,0.0
"""mean""",null,252.50726,657.331475,2.996316
"""std""",null,143.025544,526.223968,1.154295
"""min""","""Beauty""",5.01,3.84,1.0
"""25%""",null,127.84,240.32,2.0
"""50%""",null,252.97,505.44,3.0
"""75%""",null,376.33,968.96,4.0
"""max""","""Sports""",499.99,2499.55,5.0


In [15]:

# 1. Forzamos la columna a String puro para 'desbloquearla'
df_modelJ = df_modelJ.with_columns(
    pl.col("product_category").cast(pl.String)
)

# 2. Creamos el índice numérico (0, 1, 2...)
df_modelJ = df_modelJ.with_columns(
    pl.col("product_category").rank("dense").alias("cat_index").cast(pl.Int64) - 1
)

# Para que verifiques qué número le tocó a cada una:
print(df_modelJ.select(["product_category", "cat_index"]).unique().sort("cat_index"))

shape: (6, 2)
┌──────────────────┬───────────┐
│ product_category ┆ cat_index │
│ ---              ┆ ---       │
│ str              ┆ i64       │
╞══════════════════╪═══════════╡
│ Beauty           ┆ 0         │
│ Books            ┆ 1         │
│ Electronics      ┆ 2         │
│ Fashion          ┆ 3         │
│ Home & Kitchen   ┆ 4         │
│ Sports           ┆ 5         │
└──────────────────┴───────────┘


In [16]:
# esta función es mágica, se toman los valores de la columna total_revenue para transformalos en números pequeños, detrás de está función hay formulas
# por ejemplo 1.000.000 a diferencia de 1.000 nos arrojaria números similares, por ejemplo 2.45 y 4.67, sin estos valores similares el modelo
# no entenderia, se sesgaria y confiaria más en 1.000.000 que en 
df_modelJ = df_modelJ.with_columns(
    total_revenue_log = pl.col("total_revenue").log1p()
    
)

# Ahora imprimimos las primeras 5 filas
print(df_modelJ.head(5))

shape: (5, 6)
┌──────────────────┬────────┬───────────────┬────────┬───────────┬───────────────────┐
│ product_category ┆ price  ┆ total_revenue ┆ rating ┆ cat_index ┆ total_revenue_log │
│ ---              ┆ ---    ┆ ---           ┆ ---    ┆ ---       ┆ ---               │
│ str              ┆ f64    ┆ f64           ┆ f64    ┆ i64       ┆ f64               │
╞══════════════════╪════════╪═══════════════╪════════╪═══════════╪═══════════════════╡
│ Books            ┆ 128.75 ┆ 463.52        ┆ 3.5    ┆ 1         ┆ 6.141005          │
│ Fashion          ┆ 302.6  ┆ 1210.4        ┆ 3.7    ┆ 3         ┆ 7.099532          │
│ Sports           ┆ 495.8  ┆ 793.28        ┆ 4.4    ┆ 5         ┆ 6.677436          │
│ Books            ┆ 371.95 ┆ 1264.64       ┆ 5.0    ┆ 1         ┆ 7.143333          │
│ Beauty           ┆ 201.68 ┆ 806.72        ┆ 4.6    ┆ 0         ┆ 6.694215          │
└──────────────────┴────────┴───────────────┴────────┴───────────┴───────────────────┘


In [17]:
df_modelJ = df_modelJ.with_columns([
    # Estandarizamos el precio (Z-score)
    ((pl.col("price") - pl.col("price").mean()) / pl.col("price").std()).alias("price_scaled"),
    
    # Estandarizamos las reseñas
     ((pl.col("rating") - pl.col("rating").mean()) / pl.col("rating").std()).alias("rating_scaled")
  
])

print(df_modelJ)

shape: (50_000, 8)
┌─────────────┬────────┬─────────────┬────────┬───────────┬─────────────┬─────────────┬────────────┐
│ product_cat ┆ price  ┆ total_reven ┆ rating ┆ cat_index ┆ total_reven ┆ price_scale ┆ rating_sca │
│ egory       ┆ ---    ┆ ue          ┆ ---    ┆ ---       ┆ ue_log      ┆ d           ┆ led        │
│ ---         ┆ f64    ┆ ---         ┆ f64    ┆ i64       ┆ ---         ┆ ---         ┆ ---        │
│ str         ┆        ┆ f64         ┆        ┆           ┆ f64         ┆ f64         ┆ f64        │
╞═════════════╪════════╪═════════════╪════════╪═══════════╪═════════════╪═════════════╪════════════╡
│ Books       ┆ 128.75 ┆ 463.52      ┆ 3.5    ┆ 1         ┆ 6.141005    ┆ -0.865281   ┆ 0.436356   │
│ Fashion     ┆ 302.6  ┆ 1210.4      ┆ 3.7    ┆ 3         ┆ 7.099532    ┆ 0.350236    ┆ 0.609622   │
│ Sports      ┆ 495.8  ┆ 793.28      ┆ 4.4    ┆ 5         ┆ 6.677436    ┆ 1.701044    ┆ 1.216053   │
│ Books       ┆ 371.95 ┆ 1264.64     ┆ 5.0    ┆ 1         ┆ 7.143333    

In [18]:
print(df_modelJ)

shape: (50_000, 8)
┌─────────────┬────────┬─────────────┬────────┬───────────┬─────────────┬─────────────┬────────────┐
│ product_cat ┆ price  ┆ total_reven ┆ rating ┆ cat_index ┆ total_reven ┆ price_scale ┆ rating_sca │
│ egory       ┆ ---    ┆ ue          ┆ ---    ┆ ---       ┆ ue_log      ┆ d           ┆ led        │
│ ---         ┆ f64    ┆ ---         ┆ f64    ┆ i64       ┆ ---         ┆ ---         ┆ ---        │
│ str         ┆        ┆ f64         ┆        ┆           ┆ f64         ┆ f64         ┆ f64        │
╞═════════════╪════════╪═════════════╪════════╪═══════════╪═════════════╪═════════════╪════════════╡
│ Books       ┆ 128.75 ┆ 463.52      ┆ 3.5    ┆ 1         ┆ 6.141005    ┆ -0.865281   ┆ 0.436356   │
│ Fashion     ┆ 302.6  ┆ 1210.4      ┆ 3.7    ┆ 3         ┆ 7.099532    ┆ 0.350236    ┆ 0.609622   │
│ Sports      ┆ 495.8  ┆ 793.28      ┆ 4.4    ┆ 5         ┆ 6.677436    ┆ 1.701044    ┆ 1.216053   │
│ Books       ┆ 371.95 ┆ 1264.64     ┆ 5.0    ┆ 1         ┆ 7.143333    

In [19]:
df_Modelps = [
    "cat_index",
    "price_scaled",
    "rating_scaled",
    "total_revenue_log"
]

df_Inte = df_modelJ.select(df_Modelps)

print(df_Inte)

shape: (50_000, 4)
┌───────────┬──────────────┬───────────────┬───────────────────┐
│ cat_index ┆ price_scaled ┆ rating_scaled ┆ total_revenue_log │
│ ---       ┆ ---          ┆ ---           ┆ ---               │
│ i64       ┆ f64          ┆ f64           ┆ f64               │
╞═══════════╪══════════════╪═══════════════╪═══════════════════╡
│ 1         ┆ -0.865281    ┆ 0.436356      ┆ 6.141005          │
│ 3         ┆ 0.350236     ┆ 0.609622      ┆ 7.099532          │
│ 5         ┆ 1.701044     ┆ 1.216053      ┆ 6.677436          │
│ 1         ┆ 0.835115     ┆ 1.735851      ┆ 7.143333          │
│ 0         ┆ -0.355372    ┆ 1.389319      ┆ 6.694215          │
│ …         ┆ …            ┆ …             ┆ …                 │
│ 0         ┆ -1.576762    ┆ -0.516606     ┆ 4.912287          │
│ 0         ┆ 0.291715     ┆ 0.089825      ┆ 7.189205          │
│ 2         ┆ 0.696398     ┆ 0.089825      ┆ 6.894589          │
│ 0         ┆ 0.384776     ┆ -1.036404     ┆ 5.680719          │
│ 4   

In [20]:
# Tomamos el 40% de forma aleatoria
df_sample = df_Inte.sample(fraction=0.4, seed=42) 

print(f"Filas originales: {len(df_Inte)}")
print(f"Filas para el modelo: {len(df_sample)}")

Filas originales: 50000
Filas para el modelo: 20000


In [ ]:
import pymc as pm
import numpy as np
import arviz as az

# 1. EL RECORTE CRÍTICO (Ponlo aquí)
# Reducimos a 2000 filas para que el cálculo sea instantáneo


# 1. Aseguramos datos limpios de tu muestra
y_vals = np.array(df_sample["total_revenue_log"].to_list(), dtype=np.float64)
p_vals = np.array(df_sample["price_scaled"].to_list(), dtype=np.float64)
r_vals = np.array(df_sample["rating_scaled"].to_list(), dtype=np.float64)
cat_raw = df_sample["cat_index"].to_list()
cat_names, cat_idx = np.unique(cat_raw, return_inverse=True)

# --- EL MODELO ---
with pm.Model() as model_turbo:
    # Priors
    mu_a = pm.Normal("mu_a", 0, 1)
    sigma_a = pm.HalfNormal("sigma_a", 1)
    a_cat = pm.Normal("a_cat", mu=mu_a, sigma=sigma_a, shape=len(cat_names))
    
    b_p = pm.Normal("b_p", 0, 1)
    b_r = pm.Normal("b_r", 0, 1)
    
    # Media
    mu = a_cat[cat_idx] + b_p * p_vals + b_r * r_vals
    sigma = pm.HalfNormal("sigma", 1)
    
    y_like = pm.Normal("y_like", mu=mu, sigma=sigma, observed=y_vals)

    # --- AQUÍ ESTÁ EL TRUCO PARA LOS 4 MINUTOS ---
    print("Iniciando ajuste rápido (ADVI)...")
    # En lugar de pm.sample, usamos pm.fit
    approx = pm.fit(n=200, method="advi", callbacks=[pm.callbacks.CheckParametersConvergence()])
    
    # Generamos muestras desde ese ajuste para poder graficar
    idata = approx.sample(1000)

# --- GRÁFICAS ---
print("¡Listo! Generando gráficas...")
az.plot_forest(idata, var_names=["a_cat"], combined=True, figsize=(10, 8))
az.plot_posterior(idata, var_names=["b_p", "b_r"])

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install gxx`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.


NameError: name 'df_sample' is not defined